In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
path = "/content/drive/MyDrive/Dataset/2_Personal_Fitness_Tracker.csv" # Changed '-' to '=' for assignment
df = pd.read_csv(path)

1.Activity Tracking

In [3]:
import pandas as pd
path = "/content/drive/MyDrive/Dataset/2_Personal_Fitness_Tracker.csv" # Changed '-' to '=' for assignment
df = pd.read_csv(path)  # This line defines df
# Now you can call log_activity
from datetime import date

def log_activity(df, path):
    """Logs a fitness activity and updates the CSV file."""

    preset_activities = {
        "Running": {"duration": 30, "calories": 300},
        "Cycling": {"duration": 45, "calories": 400},
        "Yoga": {"duration": 60, "calories": 250},
    }

    today = date.today().strftime("%Y-%m-%d")

    activity_name = input("Enter activity name (or type 'custom'): ")

    if activity_name.lower() == 'custom':
        activity_name = input("Enter custom activity name: ")
        duration = int(input("Enter duration (minutes): "))
        calories = int(input("Enter calories burned: "))
    elif activity_name in preset_activities:
        duration = preset_activities[activity_name]["duration"]
        calories = preset_activities[activity_name]["calories"]
        print(f"Using preset values for {activity_name}: Duration - {duration} minutes, Calories - {calories}")
        duration_input = input(f"Enter duration (minutes, press Enter to use default {duration}): ")
        if duration_input:
          duration = int(duration_input)
        calories_input = input(f"Enter calories burned (press Enter to use default {calories}): ")
        if calories_input:
          calories = int(calories_input)
    else:
        print("Invalid preset activity. Please enter a valid activity or choose 'custom'.")
        return

    new_data = pd.DataFrame({
        "Date": [today],
        "Activity Name": [activity_name],
        "Duration (minutes)": [duration],
        "Calories Burned": [calories]
    })

    updated_df = pd.concat([df, new_data], ignore_index=True)
    updated_df.to_csv(path, index=False)
    print("Activity logged successfully!")

# Example usage (assuming df and path are already defined from your initial code)
log_activity(df, path) # This call now has access to df

Enter activity name (or type 'custom'): 'custom'
Invalid preset activity. Please enter a valid activity or choose 'custom'.


2.Progress Monitoring

In [ ]:
from datetime import datetime, timedelta

def weekly_summary(df):
    """Displays weekly summary of total activity duration and calories burned."""
    today = datetime.today()
    start_of_week = today - timedelta(days=today.weekday())
    end_of_week = start_of_week + timedelta(days=6)

    # Convert 'Date' column to datetime objects for comparison
    df['Date'] = pd.to_datetime(df['Date'])

    weekly_data = df[(df['Date'] >= start_of_week) & (df['Date'] <= end_of_week)]
    total_duration = weekly_data['Duration (minutes)'].sum()
    total_calories = weekly_data['Calories Burned'].sum()

    print(f"Weekly Summary ({start_of_week.strftime('%Y-%m-%d')} to {end_of_week.strftime('%Y-%m-%d')}):")
    print(f"Total Activity Duration: {total_duration} minutes")
    print(f"Total Calories Burned: {total_calories}")


def monthly_trends(df):
    """Displays monthly activity trends using a summary table."""
    # Convert 'Date' column to datetime objects
    df['Date'] = pd.to_datetime(df['Date'])
    # Extract month and year
    df['Month'] = df['Date'].dt.to_period('M')
    # Group by month and calculate monthly sums
    monthly_activity = df.groupby('Month').agg({'Duration (minutes)': 'sum', 'Calories Burned': 'sum'})
    print("\nMonthly Trends:")
    print(monthly_activity)

def compare_with_goals(df):
    """Compares actual activity duration with weekly goals."""
    weekly_goal = int(input("Enter your weekly activity goal (minutes): ")) # Get the goal from the user
    today = datetime.today()
    start_of_week = today - timedelta(days=today.weekday())
    end_of_week = start_of_week + timedelta(days=6)
    df['Date'] = pd.to_datetime(df['Date'])
    weekly_data = df[(df['Date'] >= start_of_week) & (df['Date'] <= end_of_week)]
    total_duration = weekly_data['Duration (minutes)'].sum()
    difference = total_duration - weekly_goal

    print("\nWeekly Goal Comparison:")
    print(f"Your weekly goal: {weekly_goal} minutes")
    print(f"Your total activity this week: {total_duration} minutes")
    if difference >=0 :
      print(f"You exceeded your goal by {difference} minutes!")
    else:
      print(f"You are {abs(difference)} minutes away from reaching your goal.")


#Example usage:
weekly_summary(df)
monthly_trends(df)
compare_with_goals(df)

Weekly Summary (2025-01-27 to 2025-02-02):
Total Activity Duration: 0 minutes
Total Calories Burned: 0

Monthly Trends:
         Duration (minutes)  Calories Burned
Month                                       
2025-01                3175            24400


3.Goal Setting

In [ ]:
import pandas as pd
from datetime import date, datetime, timedelta

# ... (Existing code for log_activity, weekly_summary, monthly_trends) ...

def set_goal(goals):
    """Allows the user to set or update fitness goals."""
    goal_type = input("Enter goal type ('weekly duration', 'daily calories', or 'q' to quit): ").lower()

    while goal_type != 'q':
        if goal_type == 'weekly duration':
            goals['weekly_duration'] = int(input("Enter your weekly activity goal (minutes): "))
        elif goal_type == 'daily calories':
            goals['daily_calories'] = int(input("Enter your daily calorie burn goal: "))
        else:
            print("Invalid goal type. Please enter 'weekly duration', 'daily calories', or 'q'.")

        goal_type = input("Enter another goal type or 'q' to quit: ").lower()

    print("Goals updated successfully!")
    return goals


def check_goals(df, goals):
    """Checks progress against set goals and notifies the user."""
    today = datetime.today()

    # Check weekly duration goal
    if 'weekly_duration' in goals:
        start_of_week = today - timedelta(days=today.weekday())
        end_of_week = start_of_week + timedelta(days=6)
        df['Date'] = pd.to_datetime(df['Date'])
        weekly_data = df[(df['Date'] >= start_of_week) & (df['Date'] <= end_of_week)]
        total_duration = weekly_data['Duration (minutes)'].sum()

        if total_duration >= goals['weekly_duration']:
            print("Congratulations! You've met your weekly duration goal!")
        else:
            remaining = goals['weekly_duration'] - total_duration
            print(f"You're {remaining} minutes away from your weekly duration goal.")

    # Check daily calorie goal
    if 'daily_calories' in goals:
        df['Date'] = pd.to_datetime(df['Date'])
        today_data = df[df['Date'].dt.date == today.date()]  # Filter data for today
        total_calories = today_data['Calories Burned'].sum()

        if total_calories >= goals['daily_calories']:
            print("Congratulations! You've met your daily calorie goal!")
        else:
            remaining = goals['daily_calories'] - total_calories
            print(f"You're {remaining} calories away from your daily calorie goal.")


# Initialize an empty dictionary to store goals
goals = {}

# Allow the user to set goals
goals = set_goal(goals)

# Example usage (after logging activities)
check_goals(df, goals)

Enter goal type ('weekly duration', 'daily calories', or 'q' to quit): 'weekly duration'
Invalid goal type. Please enter 'weekly duration', 'daily calories', or 'q'.
Enter another goal type or 'q' to quit: q
Goals updated successfully!


4.Advanced features



BMI Calculator

In [ ]:
import pandas as pd
from datetime import date, datetime, timedelta

# ... (Existing code for log_activity, weekly_summary, monthly_trends, set_goal, check_goals) ...

def calculate_bmi(height, weight):
    """Calculates BMI using height (in meters) and weight (in kilograms)."""
    bmi = weight / (height ** 2)
    return bmi

def suggest_activity_plan(bmi):
    """Suggests an activity plan based on BMI."""
    if bmi < 18.5:
        return "Focus on strength training and healthy calorie intake to gain weight."
    elif 18.5 <= bmi < 25:
        return "Maintain a balanced routine with a mix of cardio and strength training."
    elif 25 <= bmi < 30:
        return "Increase cardio and consider moderate calorie restriction for weight loss."
    else:
        return "Consult a doctor or nutritionist for a personalized weight management plan."

def bmi_calculator():
    """Calculates BMI and suggests an activity plan."""
    height = float(input("Enter your height in meters: "))
    weight = float(input("Enter your weight in kilograms: "))
    age = int(input("Enter your age: "))  # Age is not used in BMI calculation but can be used for further personalization

    bmi = calculate_bmi(height, weight)

    print(f"Your BMI is: {bmi:.2f}")
    print("Activity Plan Suggestion:")
    print(suggest_activity_plan(bmi))


Health Tips

In [ ]:
import pandas as pd
from datetime import date, datetime, timedelta
import random

# ... (Existing code for other functions) ...

def get_health_tip():
    """Provides a random health or fitness tip."""
    health_tips = [
        "Drink plenty of water throughout the day to stay hydrated.",
        "Get at least 7-8 hours of sleep each night for optimal recovery.",
        "Incorporate strength training into your routine 2-3 times per week.",
        "Eat a balanced diet with plenty of fruits, vegetables, and lean protein.",
        "Take breaks throughout the day to stretch and move around.",
        "Listen to your body and rest when you need it.",
        "Find an activity you enjoy and make it a regular part of your routine.",
        "Set realistic goals and celebrate your progress.",
        "Vary your workouts to challenge different muscle groups.",
        "Stay consistent with your fitness routine for long-term benefits."
    ]

    return random.choice(health_tips)

Exercise recommendations

In [ ]:
import pandas as pd
from datetime import date, datetime, timedelta
import random

# ... (Existing code for other functions) ...

def suggest_exercises(criteria, value):
    """Suggests exercises based on available time or calorie targets."""
    exercises = {
        "short_time": [
            {"name": "High-Intensity Interval Training (HIIT)", "duration": 15, "calories": 200},
            {"name": "Jumping Jacks", "duration": 10, "calories": 100},
            {"name": "Burpees", "duration": 10, "calories": 150}
        ],
        "moderate_time": [
            {"name": "Running", "duration": 30, "calories": 300},
            {"name": "Cycling", "duration": 45, "calories": 400},
            {"name": "Swimming", "duration": 30, "calories": 250}
        ],
        "long_time": [
            {"name": "Hiking", "duration": 60, "calories": 450},
            {"name": "Yoga", "duration": 60, "calories": 250},
            {"name": "Dancing", "duration": 60, "calories": 350}
        ],
        "calorie_target": [
            {"name": "Running", "duration": 30, "calories": 300},
            {"name": "Cycling", "duration": 45, "calories": 400},
            {"name": "Swimming", "duration": 30, "calories": 250},  # Adjust calories as needed
            {"name": "Boxing", "duration": 60, "calories": 600}  # Add more calorie-burning exercises
        ]
    }

    if criteria == "time":
        if value <= 20:
            suggested_exercises = exercises["short_time"]
        elif value <= 45:
            suggested_exercises = exercises["moderate_time"]
        else:
            suggested_exercises = exercises["long_time"]
    elif criteria == "calories":
        suggested_exercises = [ex for ex in exercises["calorie_target"] if ex["calories"] >= value]
    else:
        return "Invalid criteria. Please enter 'time' or 'calories'."

    if suggested_exercises:
        print(f"Suggested Exercises ({criteria} = {value}):")
        for ex in suggested_exercises:
            print(f"- {ex['name']} ({ex['duration']} minutes, {ex['calories']} calories)")
    else:
        print("No exercises found matching your criteria.")


Data Visualization

In [ ]:
import pandas as pd
from datetime import date, datetime, timedelta
import random
import matplotlib.pyplot as plt

# ... (Your existing code for other functions) ...

def visualize_weekly_activity(df):
    """Visualizes weekly activity duration using a bar chart."""
    today = datetime.today()
    start_of_week = today - timedelta(days=today.weekday())
    end_of_week = start_of_week + timedelta(days=6)

    df['Date'] = pd.to_datetime(df['Date'])
    weekly_data = df[(df['Date'] >= start_of_week) & (df['Date'] <= end_of_week)]

    # Group by day of the week and sum duration
    daily_activity = weekly_data.groupby(weekly_data['Date'].dt.day_name())['Duration (minutes)'].sum()


    plt.figure(figsize=(8, 6))
    plt.bar(daily_activity.index, daily_activity.values)
    plt.title('Weekly Activity Duration')
    plt.xlabel('Day of the Week')
    plt.ylabel('Total Duration (minutes)')
    plt.show()


def visualize_monthly_calories(df):
    """Visualizes monthly calorie burn using a line chart."""
    df['Date'] = pd.to_datetime(df['Date'])
    df['Month'] = df['Date'].dt.to_period('M')
    monthly_calories = df.groupby('Month')['Calories Burned'].sum()

    plt.figure(figsize=(10, 6))
    plt.plot(monthly_calories.index.astype(str), monthly_calories.values)  # Convert PeriodIndex to strings
    plt.title('Monthly Calories Burned')
    plt.xlabel('Month')
    plt.ylabel('Total Calories Burned')
    plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
    plt.tight_layout()  # Adjust layout to prevent labels from overlapping
    plt.show()

5.User interface

In [ ]:
import pandas as pd
from datetime import date, datetime, timedelta
import random
import matplotlib.pyplot as plt

# ... (Your existing code for log_activity, weekly_summary,
# monthly_trends, set_goal, check_goals, calculate_bmi,
# suggest_activity_plan, bmi_calculator, get_health_tip,
# suggest_exercises, visualize_weekly_activity,
# visualize_monthly_calories) ...

def main():
    """Main function to run the fitness tracker."""
    path = "/content/drive/MyDrive/Dataset/2_Personal_Fitness_Tracker.csv"
    df = pd.read_csv(path)
    goals = {}  # Initialize goals dictionary

    while True:
        print("\nFitness Tracker Menu:")
        print("1. Log a new activity")
        print("2. View activity history")
        print("3. Set and update fitness goals")
        print("4. View progress reports")
        print("5. BMI Calculator")
        print("6. Get a health tip")
        print("7. Get exercise recommendations")
        print("8. Visualize weekly activity")
        print("9. Visualize monthly calories")
        print("0. Exit")

        choice = input("Enter your choice: ")

        if choice == '1':
            log_activity(df, path)
            df = pd.read_csv(path)  # Reload data after logging
        elif choice == '2':
            print("\nActivity History:")
            print(df)
        elif choice == '3':
            goals = set_goal(goals)
        elif choice == '4':
            check_goals(df, goals)
            weekly_summary(df)
            monthly_trends(df)
        elif choice == '5':
            bmi_calculator()
        elif choice == '6':
            print("\nHealth Tip:", get_health_tip())
        elif choice == '7':
            criteria = input("Enter criteria ('time' or 'calories'): ").lower()
            value = int(input("Enter value (time in minutes or calorie target): "))
            suggest_exercises(criteria, value)
        elif choice == '8':
            visualize_weekly_activity(df)
        elif choice == '9':
            visualize_monthly_calories(df)
        elif choice == '0':
            print("Exiting fitness tracker.")
            break
        else:
            print("Invalid choice. Please try again.")

